# Quantum State Tomography

In [1]:
import qiskit
from qiskit_experiments.framework import ParallelExperiment
from qiskit_experiments.library import StateTomography

# For simulation
from qiskit.providers.aer import AerSimulator
from qiskit.test.mock import FakeParis

# Noisy simulator backend
backend = AerSimulator.from_backend(FakeParis())

## State Tomography Experiment

To run a state tomography experiment we initialize the experiment with a circuit to prepare the state to be measured. We can also pass in an `Operator`, or a `Statevector` to describe the preparation circuit.

In [2]:
# Run experiments

# GHZ State preparation circuit
nq = 2
qc_ghz = qiskit.QuantumCircuit(nq)
qc_ghz.h(0)
qc_ghz.s(0)
for i in range(1, nq):
    qc_ghz.cx(0, i)

# QST Experiment
qstexp1 = StateTomography(qc_ghz)
qstdata1 = qstexp1.run(backend, seed_simulation=100).block_for_results()

# Print results
for result in qstdata1.analysis_results():
    print(result)

DbAnalysisResultV1
- name: state
- value: DensityMatrix([[ 0.48421224+0.j        , -0.00309245+0.00309245j,
                 0.01676432+0.00048828j,  0.01660156-0.44775391j],
               [-0.00309245-0.00309245j,  0.02750651+0.j        ,
                -0.00390625+0.00537109j,  0.00309245-0.00146484j],
               [ 0.01676432-0.00048828j, -0.00390625-0.00537109j,
                 0.03011068+0.j        , -0.00797526-0.00569661j],
               [ 0.01660156+0.44775391j,  0.00309245+0.00146484j,
                -0.00797526+0.00569661j,  0.45817057+0.j        ]],
              dims=(2, 2))
- extra: <4 items>
- device_components: ['Q0', 'Q1']
- verified: False
DbAnalysisResultV1
- name: state_fidelity
- value: 0.9189453124999996
- device_components: ['Q0', 'Q1']
- verified: False
DbAnalysisResultV1
- name: positive
- value: True
- device_components: ['Q0', 'Q1']
- verified: False


### Tomography Results

The main result for tomography is the fitted state, which is stored as a `DensityMatrix` object:

In [3]:
state_result = qstdata1.analysis_results("state")
print(state_result.value)

DensityMatrix([[ 0.48421224+0.j        , -0.00309245+0.00309245j,
                 0.01676432+0.00048828j,  0.01660156-0.44775391j],
               [-0.00309245-0.00309245j,  0.02750651+0.j        ,
                -0.00390625+0.00537109j,  0.00309245-0.00146484j],
               [ 0.01676432-0.00048828j, -0.00390625-0.00537109j,
                 0.03011068+0.j        , -0.00797526-0.00569661j],
               [ 0.01660156+0.44775391j,  0.00309245+0.00146484j,
                -0.00797526+0.00569661j,  0.45817057+0.j        ]],
              dims=(2, 2))


The state fidelity of the fitted state with the ideal state prepared by the input circuit is stored in the `"state_fidelity"` result field. Note that if the input circuit contained any measurements the ideal state cannot be automatically generated and this field will be set to `None`.

In [4]:
fid_result = qstdata1.analysis_results("state_fidelity")
print("State Fidelity = {:.5f}".format(fid_result.value))

State Fidelity = 0.91895


#### Additional state metadata

Additional data is stored in the tomography under the `"state_metadata"` field. This includes
- `eigvals`: the eigenvalues of the fitted state
- `trace`: the trace of the fitted state
- `positive`: Whether the eigenvalues are all non-negative
- `positive_delta`: the deviation from positivity given by 1-norm of negative eigenvalues.

If trace rescaling was performed this dictionary will also contain a `raw_trace` field containing the trace before rescaling.
Futhermore, if the state was rescaled to be positive or trace 1 an additional field `raw_eigvals` will contain the state eigenvalues before rescaling was performed.

In [5]:
state_result.extra

{'fitter': 'linear_inversion',
 'fitter_time': 0.00232696533203125,
 'eigvals': array([0.9197649 , 0.04063239, 0.02551458, 0.01408814]),
 'trace': 1.0000000000000002}

To see the effect of rescaling we can perform a "bad" fit with very low counts

In [6]:
# QST Experiment
bad_data = qstexp1.run(backend, shots=10, seed_simulation=100).block_for_results()
bad_state_result = bad_data.analysis_results("state")

# Print result
print(bad_state_result)

# Show extra data
bad_state_result.extra

DbAnalysisResultV1
- name: state
- value: DensityMatrix([[ 0.43622746+0.00000000e+00j,  0.05138676+4.99391814e-02j,
                -0.00306491-5.52153122e-02j, -0.14554754-4.22947404e-01j],
               [ 0.05138676-4.99391814e-02j,  0.01362761+0.00000000e+00j,
                -0.0043994 +3.62881273e-05j, -0.07558927-3.67544469e-02j],
               [-0.00306491+5.52153122e-02j, -0.0043994 -3.62881273e-05j,
                 0.03044341+0.00000000e+00j,  0.03025771+1.35412073e-02j],
               [-0.14554754+4.22947404e-01j, -0.07558927+3.67544469e-02j,
                 0.03025771-1.35412073e-02j,  0.51970153+0.00000000e+00j]],
              dims=(2, 2))
- extra: <5 items>
- device_components: ['Q0', 'Q1']
- verified: False


{'fitter': 'linear_inversion',
 'fitter_time': 0.0013802051544189453,
 'eigvals': array([0.94406352, 0.05593648, 0.        , 0.        ]),
 'raw_eigvals': array([ 0.99490614,  0.10677911,  0.04411653, -0.14580179]),
 'trace': 0.9999999999999992}

## Tomography Fitters

The default fitters is `linear_inversion`, which reconstructs the state using *dual basis* of the tomography basis. This will typically result in a non-postive reconstructed state. This state is rescaled to be postive-semidfinite (PSD) by computing its eigen-decomposition and rescaling its eigenvalues using the approach from *J Smolin, JM Gambetta, G Smith, Phys. Rev. Lett. 108, 070502 (2012), [open access](https://arxiv.org/abs/arXiv:1106.5458).

There are several other fitters are included (See API documentation for details). For example if `cvxpy` is installed we can use the `cvxpy_gaussian_lstsq` fitter which allows constraining the fit to be PSD without requiring rescaling.

In [7]:
try:
    import cvxpy
    
    # Set analysis option for cvxpy fitter
    qstexp1.analysis.set_options(fitter='cvxpy_gaussian_lstsq')
    
    # Re-run experiment
    qstdata2 = qstexp1.run(backend, seed_simulation=100).block_for_results()

    state_result2 = qstdata2.analysis_results("state")
    print(state_result2)   
    print("\nextra:")
    for key, val in state_result2.extra.items():
        print(f"- {key}: {val}")

except ModuleNotFoundError:
    print("CVXPY is not installed")

DbAnalysisResultV1
- name: state
- value: DensityMatrix([[ 0.49143631+0.00000000e+00j,  0.00326385-1.03722828e-04j,
                -0.01415099-1.56676188e-02j,  0.00503769-4.44943896e-01j],
               [ 0.00326385+1.03722828e-04j,  0.02682787+0.00000000e+00j,
                 0.00956292+2.59008462e-03j,  0.01619132-3.06513469e-03j],
               [-0.01415099+1.56676188e-02j,  0.00956292-2.59008462e-03j,
                 0.02575371-4.33680869e-19j,  0.00971971-1.02442434e-02j],
               [ 0.00503769+4.44943896e-01j,  0.01619132+3.06513469e-03j,
                 0.00971971+1.02442434e-02j,  0.45598212-8.67361738e-19j]],
              dims=(2, 2))
- extra: <8 items>
- device_components: ['Q0', 'Q1']
- verified: False

extra:
- cvxpy_solver: CVXOPT
- cvxpy_status: optimal
- fitter: cvxpy_gaussian_lstsq
- fitter_time: 0.03989887237548828
- eigvals: [0.91957528 0.05046516 0.02616115 0.00379841]
- raw_eigvals: [0.91881748 0.05042357 0.02613959 0.00379528]
- trace: 1.0
- raw_trace

## Parallel Tomography Experiment

We can also use the `qiskit_experiments.ParallelExperiment` class to run subsystem tomography on multiple qubits in parallel.

For example if we want to perform 1-qubit QST on several qubits at once:

In [8]:
from math import pi
num_qubits = 5
gates = [qiskit.circuit.library.RXGate(i * pi / (num_qubits - 1))
         for i in range(num_qubits)]

subexps = [
    StateTomography(gate, qubits=[i])
    for i, gate in enumerate(gates)
]
parexp = ParallelExperiment(subexps)
pardata = parexp.run(backend, seed_simulation=100).block_for_results()

for result in pardata.analysis_results():
    print(result)

DbAnalysisResultV1
- name: parallel_experiment
- value: 5
- extra: <2 items>
- device_components: ['Q0', 'Q1', 'Q2', 'Q3', 'Q4']
- verified: False


View component experiment analysis results

In [9]:
for i, expdata in enumerate(pardata.child_data()):
    state_result_i = expdata.analysis_results("state")
    fid_result_i = expdata.analysis_results("state_fidelity")
    
    print(f'\nPARALLEL EXP {i}')
    print("State Fidelity: {:.5f}".format(fid_result_i.value))
    print("State: {}".format(state_result_i.value))


PARALLEL EXP 0
State Fidelity: 0.98828
State: DensityMatrix([[ 0.98828125+0.j        , -0.00195312-0.03515625j],
               [-0.00195312+0.03515625j,  0.01171875+0.j        ]],
              dims=(2,))

PARALLEL EXP 1
State Fidelity: 0.96887
State: DensityMatrix([[0.84667969+0.j        , 0.04785156+0.31640625j],
               [0.04785156-0.31640625j, 0.15332031+0.j        ]],
              dims=(2,))

PARALLEL EXP 2
State Fidelity: 0.97656
State: DensityMatrix([[0.50683594+0.j       , 0.01171875+0.4765625j],
               [0.01171875-0.4765625j, 0.49316406+0.j       ]],
              dims=(2,))

PARALLEL EXP 3
State Fidelity: 0.96542
State: DensityMatrix([[0.16699219+0.j        , 0.01269531+0.32519531j],
               [0.01269531-0.32519531j, 0.83300781+0.j        ]],
              dims=(2,))

PARALLEL EXP 4
State Fidelity: 0.97070
State: DensityMatrix([[0.02929688+0.j        , 0.        +0.01171875j],
               [0.        -0.01171875j, 0.97070312+0.j        ]],
          

In [10]:
import qiskit.tools.jupyter
%qiskit_copyright